<a href="https://colab.research.google.com/github/DiegoMeloCoder/Machine_Learning_T3Profundizacion/blob/main/modeloRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Conexion

In [ ]:
from os import read
import sqlite3
import pandas as pd

# Conexion
conexion = sqlite3.connect('datos_sensores.db')

# Codigo sql para hacer consulta
query = """

SELECT

    datos_basicos.tamaño,
    datos_basicos.categoria,
    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NOT NULL AND clasificacion.etiqueta <> '';

"""
dataset = pd.read_sql_query(query,conexion)


Creacion y Entrenamiento del modelo con Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix;
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = dataset.drop(['segmento_numerico','id','tamaño','categoria'], axis=1)
Y = dataset['segmento_numerico']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Normalización de datos
scaler = StandardScaler()
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.transform(X_Test)

# Manejo de desequilibrio de clases con SMOTE
smote = SMOTE(random_state=1)
X_Train_resampled, Y_Train_resampled = smote.fit_resample(X_Train_scaled, Y_Train)


In [ ]:
# Modelo RandomForest
modelo_rf = RandomForestClassifier( n_estimators=120,  # Puedes ajustar este valor
    max_depth=50,    # Puedes ajustar este valor
    min_samples_split=2,  # Puedes ajustar este valor
    min_samples_leaf=1,   # Puedes ajustar este valor
    random_state=1)
modelo_rf.fit(X_Train_resampled, Y_Train_resampled)

# Predicciones y evaluación del modelo
predicciones_rf = modelo_rf.predict(X_Test_scaled)

# accuracy

accuracy= accuracy_score(Y_Test,predicciones_rf)
accuracy

0.934

In [ ]:
 # query que selecciona los datos faltantes
 query2 = """

SELECT
datos_basicos.tamaño,
    datos_basicos.categoria,
    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
	FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NULL OR clasificacion.etiqueta = '';

 """

 dataset2 = pd.read_sql_query(query2,conexion)

# Predictores de las variables a rellenar
X_completar = dataset2.drop(['segmento_numerico', 'id','tamaño','categoria'],axis=1)

#Normalizacion
X_completar_scaled = scaler.transform(X_completar)

In [ ]:
# Variable que contiene los datos faltantes
Datos_Faltantes = modelo_rf.predict(X_completar_scaled)


resultados_prediccion = pd.DataFrame({
    'id': dataset2['id'],  # Tomamos 'id' de dataset2
    'Prediccion_Modelo': Datos_Faltantes
})

# Mostrar el DataFrame con los resultados
print(resultados_prediccion)

        id  Prediccion_Modelo
0   300001                  0
1   300002                  0
2   300003                  1
3   300004                  1
4   300005                  1
..     ...                ...
95  300096                  1
96  300097                  1
97  300098                  1
98  300099                  1
99  300100                  1

[100 rows x 2 columns]


In [ ]:
resultados_prediccion.to_csv('predicciones.csv', index=False)

In [ ]:
reporte = classification_report(Y_Test,predicciones_rf)
print(reporte)

              precision    recall  f1-score   support

           0       0.97      0.90      0.93     30247
           1       0.91      0.97      0.94     29753

    accuracy                           0.93     60000
   macro avg       0.94      0.93      0.93     60000
weighted avg       0.94      0.93      0.93     60000



In [ ]:
matriz_confusion = confusion_matrix(Y_Test,predicciones_rf)
print(matriz_confusion)

[[27260  2987]
 [  973 28780]]
